In [1]:
import pandas as pd
from app.data.utils.bigquery import BigQuery
from app.conf.settings import PROJECT_ID, DATASET_ID,SECRET_NAME, DEFAULT_PARAMS
from app.data.utils.great_expectations_service  import DataQuality
from app.data.utils.load_query import load_format

2022-11-29T16:13:53-0400 - WARNING: /root/.cache/pypoetry/virtualenvs/app-T22veBjD-py3.7/lib/python3.7/site-packages/great_expectations/execution_engine/sqlalchemy_execution_engine.py:157: UserWarning: Obsolete pybigquery is installed, which is likely to
interfere with sqlalchemy_bigquery.
pybigquery should be uninstalled.
  import sqlalchemy_bigquery as sqla_bigquery
  [level: WARNING]


In [2]:
bigquery = BigQuery()


In [3]:
query = """

    WITH prints AS (
        SELECT
            id,
            event_data,
            `ds`,
            site,
             json_extract_scalar(`event_data`, '$.print_id') AS print_id,
            `user`.`user_id` AS `user_id`,
            CAST(REPLACE(`server_timestamp`, "T", ' ') AS TIMESTAMP) AS ts
        FROM
            meli-bi-data.MELIDATA.ADVERTISING
        WHERE
            `ds` >= '2022-11-28'
            AND `ds` < '{end_date}'
            AND `event` = 'display_prints'
            AND site = '{site_id}'
            AND json_extract_scalar(`event_data`, '$.content_source') = 'DSP'
            AND json_extract_scalar(event_data, '$.valid')= 'true'
            AND (NOT
                REGEXP_CONTAINS(LOWER(`device`.user_agent),
                '.*(libwww|wget|lwp|damnBot|bbbike|java|spider|crawl|slurp|bot|feedburner|googleimageproxy|google web preview).*')
            )
    ),
    clicks AS (
        SELECT
            `ds`,
            site,
            `user`.`user_id` AS `user_id`,
            json_extract_scalar(`event_data`, '$.print_id') AS print_id,
            json_extract_scalar(`event_data`, '$.click_id') AS click_id,
            CAST(REPLACE(`server_timestamp`, "T", ' ') AS TIMESTAMP) AS ts
        FROM
            meli-bi-data.MELIDATA.ADVERTISING
        WHERE
            `ds` >= '2022-11-28'
            AND `event` = 'display_clicks'
            AND site = '{site_id}'
            AND json_extract_scalar(`event_data`, '$.content_source') = 'DSP'
            AND (NOT
                REGEXP_CONTAINS(LOWER(`device`.user_agent),
                '.*(libwww|wget|lwp|damnBot|bbbike|java|spider|crawl|slurp|bot|feedburner|googleimageproxy|google web preview).*')
            )
    ),
    prints_clicks AS (
        SELECT DISTINCT
            prints.id AS id,
            prints.ds AS ds,
            prints.site,
            CAST(
                json_extract_scalar(prints.event_data, '$.campaign_id') AS INT64
            ) AS campaign_id,
            CAST(json_extract_scalar(prints.event_data, '$.line_item_id') AS INT64) AS line_item_id,
            CAST(json_extract_scalar(prints.event_data, '$.creative_id') AS INT64) AS creative_id,
            CASE
                WHEN clicks.ds IS NOT NULL THEN 1 
                ELSE 0
            END AS target
        FROM prints
        LEFT JOIN clicks
        ON prints.print_id = clicks.print_id
            AND ((prints.user_id = clicks.user_id) OR clicks.user_id IS NULL)
            AND TIMESTAMP_DIFF(clicks.ts, prints.ts, SECOND) <= {click_window}
            AND prints.site = clicks.site
    )

    SELECT
        ds,
        site,
        campaign_id,
        line_item_id,
        creative_id,
        COUNT(*) AS n_prints,
        SUM(target) AS n_clicks
    FROM prints_clicks
    GROUP BY 1,2,3,4,5
    ;
"""

In [7]:
#query = 'select * from meli-bi-data.SBOX_DSPCREATIVOS.BQ_PRINTS_CLICKS_PER_DAY'

In [4]:
params = DEFAULT_PARAMS
sql = query.format(**params)

df_great = bigquery.run_query(sql)

2022-11-29T16:14:01-0400 - INFO: Executing...  [level: INFO]


Executing query...


2022-11-29 16:14:03,292 - INFO melitk.metrics.fallback : Datadog attempt: func=record_count args=('melitk.connectors.bigquery.query_success',) kwargs={'tags': {'duration': '0', 'platform': 'not_fury', 'read_engine': 'google_cloud_bigquery'}}


In [5]:
df_great.head()

,ds,site,campaign_id,line_item_id,creative_id,n_prints,n_clicks
0,2022-11-28,MLA,3992,9406,8095,356248,859
1,2022-11-28,MLA,3991,9404,8092,350518,787
2,2022-11-28,MLA,3987,9352,8089,101466,190
3,2022-11-28,MLA,3987,9352,8088,102284,238
4,2022-11-28,MLA,3993,9407,8096,415230,740


In [6]:
df_great.site.unique()

array(['MLA'], dtype=object)

In [7]:
dq_checker = DataQuality('track','Pandas',df_great,'test')
validator = dq_checker.get_validator()

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	pandas_runtime_connector:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



In [8]:
'''validator.expect_column_values_to_be_of_type('campaign_id','int64')
validator.expect_column_values_to_be_of_type('line_item_id','int64')
validator.expect_column_values_to_be_of_type('n_prints','int64')
validator.expect_column_values_to_be_of_type('n_clicks','int64')
validator.expect_column_distinct_values_to_be_in_set('site',['MLA'])
validator.expect_column_values_to_be_between('n_prints',min_value= 0)
validator.expect_column_values_to_be_between('n_clicks',min_value= 0)'''
validator.expect_column_pair_values_A_to_be_greater_than_B('n_prints','n_clicks',or_equal=True)
validator.save_expectation_suite()

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

2022-11-29T16:16:49-0400 - INFO: 	1 expectation(s) included in expectation_suite.  [level: INFO]
2022-11-29 16:16:49,555 - INFO great_expectations.validator.validator : 	1 expectation(s) included in expectation_suite.


In [9]:
result = dq_checker.validate_data()
print(result)

2022-11-29T16:16:49-0400 - INFO: track checkpoint is already created  [level: INFO]
2022-11-29 16:16:49,687 - INFO app.data.utils.great_expectations_service : track checkpoint is already created
2022-11-29T16:16:49-0400 - INFO: 	1 expectation(s) included in expectation_suite.  [level: INFO]
2022-11-29 16:16:49,812 - INFO great_expectations.validator.validator : 	1 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{'success': True, 'success_statistics': {'evaluated_expectations': 1, 'successful_expectations': 1, 'unsuccessful_expectations': 0, 'success_percent': 100.0}, 'expectations': [{'column_A': 'n_prints', 'expectation_type': 'expect_column_pair_values_a_to_be_greater_than_b', 'success': True, 'args': {'column_B': 'n_clicks', 'or_equal': True}, 'expectation_results': {'element_count': 8, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_percent_total': 0.0, 'unexpected_percent_nonmissing': 0.0}}]}


## Revison Bigquery

In [9]:
dq_checker2 = DataQuality('track2','Bigquery',query,'test')
validator2 = dq_checker2.get_validator()

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: SqlAlchemyExecutionEngine
Data Connectors:
	bigquery_runtime_connector:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []

	default_inferred_data_connector_name : InferredAssetSqlDataConnector

	Available data_asset_names (3 of 32281):
		BIGQUERY_GOVERNANCE.BIGQUERY_COST_ATTRIBUTION (1 of 1): [{}]
		DATAMART.BT_DM_MKP_ORD_ORDER_MULTI_COUPONS_CUSTOM (1 of 1): [{}]
		DATAMART.BT_MKP_VISITS_ITEMS_CAMPAIGNS (1 of 1): [{}]

	Unmatched data_references (0 of 0):[]



2022-11-29T14:27:18-0400 - INFO: expectation_suite with name track2_expectation_suite already exists. If you would like to overwrite this expectation_suite, set overwrite_existing=True.  [level: INFO]
2022-11-29 14:27:18,122 - INFO app.data.utils.great_expectations_service : expectation_suite with name track2_expectation_suite already exists. If you would like to overwrite this expectation_suite, set overwrite_existing=True.


In [10]:
validator2.expect_column_values_to_be_of_type('site','STRING')
validator2.expect_column_values_to_be_of_type('campaign_id','INTEGER')
validator2.expect_column_values_to_be_of_type('line_item_id','INTEGER')
validator2.expect_column_values_to_be_of_type('n_prints','INTEGER')
validator2.expect_column_values_to_be_of_type('n_clicks','INTEGER')
validator2.expect_column_values_to_be_of_type('days','INTEGER')
#validator2.expect_column_distinct_values_to_be_in_set('site',['MLA'])
#validator2.expect_column_pair_values_A_to_be_greater_than_B('n_prints','n_clicks',or_equal=True)
validator2.save_expectation_suite()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

2022-11-29T14:27:21-0400 - INFO: 	6 expectation(s) included in expectation_suite.  [level: INFO]
2022-11-29 14:27:21,597 - INFO great_expectations.validator.validator : 	6 expectation(s) included in expectation_suite.


In [11]:
result = dq_checker2.validate_data()
print(result)

2022-11-29T14:27:21-0400 - INFO: track2 checkpoint is already created  [level: INFO]
2022-11-29 14:27:21,701 - INFO app.data.utils.great_expectations_service : track2 checkpoint is already created
2022-11-29T14:27:24-0400 - INFO: 	6 expectation(s) included in expectation_suite.  [level: INFO]
2022-11-29 14:27:24,001 - INFO great_expectations.validator.validator : 	6 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{'success': True, 'success_statistics': {'evaluated_expectations': 6, 'successful_expectations': 6, 'unsuccessful_expectations': 0, 'success_percent': 100.0}, 'expectations': [{'column': 'site', 'expectation_type': 'expect_column_values_to_be_of_type', 'success': True, 'args': {'type_': 'STRING'}, 'expectation_results': {'observed_value': 'String'}}, {'column': 'campaign_id', 'expectation_type': 'expect_column_values_to_be_of_type', 'success': True, 'args': {'type_': 'INTEGER'}, 'expectation_results': {'observed_value': 'Integer'}}, {'column': 'line_item_id', 'expectation_type': 'expect_column_values_to_be_of_type', 'success': True, 'args': {'type_': 'INTEGER'}, 'expectation_results': {'observed_value': 'Integer'}}, {'column': 'n_prints', 'expectation_type': 'expect_column_values_to_be_of_type', 'success': True, 'args': {'type_': 'INTEGER'}, 'expectation_results': {'observed_value': 'Integer'}}, {'column': 'n_clicks', 'expectation_type': 'expect_column_values_to_be_of_type', 'success':